In [ ]:
## Wiesner’s Quantum Money (Proof of Concept)

Stephen Wiesner invented the concept of **quantum money** in 1968, an idea that remains unimplemented in real-world quantum systems due to the lack of quantum memory, which is simulated in this Lab. The central idea is that a banknote can be represented by a qubit prepared in one of two possible bases (computational or Hadamard). Because unknown quantum states cannot be cloned (no-cloning theorem), counterfeit banknotes cannot be forged reliably. Each quantum coin is issued together with a unique **serial number**, which is recorded in the bank’s ledger. This serial number serves as the identifier that links the quantum coin held by the user to the corresponding preparation record stored by the bank.  

In Wiesner’s original proposal, the bank records only the secret preparation basis for each qubit, which is later used to verify authenticity. In this simulation, both the preparation label and the actual quantum state are stored in the bank’s ledger. This allows verification to be performed by directly comparing the fidelity of the received coin with the bank’s reference state.  

![QuantumMoney](images/QuantumMoney.png)

This proof-of-concept simulation illustrates:
- **Coin creation:** The bank randomly chooses one of four states ($\lvert 0\rangle, \lvert 1\rangle, \lvert +\rangle, \lvert -\rangle$) and stores it in the bank ledger.  
- **Wallets:** Alice and Bob each have a “wallet” to hold their quantum coins.  
- **Issuing:** Alice can request coins from the bank.  
- **Verification:** Coins are checked against the bank’s reference state using fidelity.  
- **Transfer:** Coins may be transferred between Alice and Bob if they pass verification.  

*Reference:*  
S. Wiesner, *Conjugate Coding*, SIGACT News, vol. 15, no. 1, pp. 78–88, 1983. (Originally written in 1968)  

---

### Task
1. Run the program to observe how a qubit, representing a quantum coin, is issued, stored in a wallet, and transferred.  
2. Study the functions and understand how:  
   - `prepare_coin_state` generates the qubit state,  
   - `create_quantum_coin_pair` creates the bank copy and the user copy,  
   - `verify_coin` checks authenticity using fidelity.  
3. Provide an explanation of why fidelity is necessary in verification and why a qubit cannot be copied classically.  

### Expected Output
When the program is executed, the bank issues two coins to Alice and records their serial numbers.  
Alice’s wallet initially contains both coins, while Bob’s wallet is empty.  
After a valid transfer, one coin is successfully moved from Alice’s wallet to Bob’s, with the fidelity value confirming its authenticity.  
The final state is that Alice retains one coin and Bob holds the transferred coin.  

If tampering or errors are introduced before verification, the fidelity drops below the acceptance threshold.  
In this case, the transfer attempt fails, the coin is rejected, and it remains recorded in Alice’s wallet.  

### Experimentation
Possible extensions of the program include:  
- Issuing additional coins and observing how unique serial numbers are generated.  
- Introducing deliberate tampering by applying random gates (e.g., `x`, `h`) to user coins before verification, and checking how fidelity changes.  
- Adjusting the fidelity threshold (currently set to `0.99`) to explore its effect on acceptance or rejection.  
- Allowing Bob to request coins directly from the bank.  
- Extending the system to **multi-qubit coins** by modifying `prepare_coin_state` to handle more than one qubit.  
- Recording all transactions in a log to simulate a basic blockchain-style ledger for quantum money.  

---

**Reflection question:**  
Why does the no-cloning theorem make it impossible for a counterfeiter to copy a quantum coin with perfect fidelity?  


In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, state_fidelity
import random

# -- Bank ledger and user wallets --
bank_ledger  = {}
Alice_wallet = {}
Bob_wallet   = {}

# -- Quantum state preparation --
def prepare_coin_state(prep):
    """
    Given a string label ('0', '1', '+', '-'), 
    return the corresponding single-qubit Statevector.
    """
    qc = QuantumCircuit(1)
    if prep == '0':
        pass
    elif prep == '1':
        qc.x(0)
    elif prep == '+':
        qc.h(0)
    elif prep == '-':
        qc.x(0)
        qc.h(0)
    state = Statevector(qc)
    return state

# -- Quantum coin creation --
def create_quantum_coin_pair():
    """
    One copy of the state is kept by the bank; 
    one copy is issued to the user.
    """
    # Step 1: Randomly choose one of the four states.
    states = ['0', '1', '+', '-']
    chosen_state = random.choice(states)
    
    # Step 2: Prepare the quantum coin.
    state = prepare_coin_state(chosen_state)
    
    # Step 3: Generate a unique serial number and store it in the bank ledger.
    while (serial := random.randint(1000, 9999)) in bank_ledger:
        continue

    bank_ledger[serial] = {
        'state': state,
        'preparation': chosen_state
    }

    # Step 4: Regenerate the state with the same preparation (user copy).
    state = prepare_coin_state(chosen_state)
    return serial, state

# -- Request a coin from the bank --
def request_coin(user_ledger):
    serial, state = create_quantum_coin_pair()
    user_ledger[serial] = state
    return serial

# Get the list of coin serial numbers in a user's wallet.
def list_user_coins(wallet):
    return list(wallet.keys())
    
# -- Coin verification --
def verify_coin(serial, user_state):
    """
    Compare the user's state to the bank's reference 
    using state fidelity (threshold 0.99).
    """
    if serial not in bank_ledger:
        return False, "Invalid serial number"
    
    bank_state = bank_ledger[serial]['state']
    fidelity = state_fidelity(bank_state, user_state)
    return fidelity >= 0.99, fidelity

# -- Transfer a coin between users --
def transfer_coin(serial, from_ledger, to_ledger):
    if serial not in from_ledger:
        print("Transfer failed: coin not found in sender's ledger.")
        return False
    
    user_coin = from_ledger[serial]
    valid, fidelity = verify_coin(serial, user_coin)
    
    if valid:
        # Move coin.
        to_ledger[serial] = user_coin
        del from_ledger[serial]
        print(f"Transfer successful. Fidelity: {fidelity:.4f}")
        return True
    else:
        print(f"Transfer failed: Fidelity: {fidelity:.4f}")
        return False

# ------------------------------------------------
#                main program
# ------------------------------------------------

# Alice requests two coins.
serial_1 = request_coin(Alice_wallet)
print(f"Coin {serial_1} issued.")
serial_2 = request_coin(Alice_wallet)
print(f"Coin {serial_2} issued.")

# Alice and Bob list their coins.
coins = list_user_coins(Alice_wallet)
print(f"Alice's coins: {coins}")

coins = list_user_coins(Bob_wallet)
print(f"Bob's coins: {coins}")

# Alice transfers a coin to Bob.
transfer_status = transfer_coin(serial_2, Alice_wallet, Bob_wallet)

# Alice and Bob list their coins again.
coins = list_user_coins(Alice_wallet)
print(f"Alice's coins: {coins}")

coins = list_user_coins(Bob_wallet)
print(f"Bob's coins: {coins}")
